### Test the yahoo finance

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import set_style

## import model objects
from sklearn.model_selection import train_test_split
import yfinance as yf
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [3]:
df= pd.read_csv("../Data/dataset_others_class.csv")

In [6]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


In [4]:
df.dtypes

Date                   object
CADUSD=X_Open         float64
CADUSD=X_Close        float64
CADUSD=X_High         float64
CADUSD=X_Low          float64
                       ...   
F_ROC                 float64
F_PVT                 float64
F_Bollinger_Upper     float64
F_Bollinger_Lower     float64
F_Close_pred_class      int64
Length: 155, dtype: object

In [7]:
df.describe()

,CADUSD=X_Open,CADUSD=X_Close,CADUSD=X_High,CADUSD=X_Low,CADUSD=X_Volume,CADUSD=X_Change,CADUSD=X_Gain,CADUSD=X_Loss,CADUSD=X_Avg_Gain,CADUSD=X_Avg_Loss,...,F_RSI,F_k_percent,F_r_percent,F_MACD,F_MACD_EMA,F_ROC,F_PVT,F_Bollinger_Upper,F_Bollinger_Lower,F_Close_pred_class
count,1237.000000,1237.000000,1237.000000,1237.000000,1237.0,1237.000000,1237.000000,1237.000000,1237.000000,1237.000000,...,1237.000000,1237.000000,1237.000000,1237.000000,1237.000000,1237.000000,1.237000e+03,1237.000000,1237.000000,1237.000000
mean,0.758633,0.758637,0.760732,0.756380,0.0,-0.000026,0.001221,0.001247,0.001222,0.001246,...,50.709762,49.819518,-50.180482,0.010772,0.010580,0.740316,9.941460e+07,13.119785,10.870765,0.489086
std,0.028260,0.028256,0.028313,0.028251,0.0,0.003274,0.001931,0.001986,0.000539,0.000647,...,17.235668,30.409753,30.409753,0.421543,0.395604,10.283284,6.203849e+07,3.942885,3.134942,0.500083
min,0.690026,0.689741,0.693371,0.681826,0.0,-0.012518,0.000000,0.000000,0.000048,0.000016,...,8.700013,0.000000,-100.000000,-1.071207,-0.953758,-40.504446,-8.287094e+07,5.333265,3.663582,0.000000
25%,0.736106,0.736106,0.738007,0.734338,0.0,-0.001900,0.000000,0.000000,0.000861,0.000809,...,39.339368,22.988487,-77.011513,-0.208349,-0.203198,-5.059206,6.629778e+07,11.077180,9.076939,0.000000
50%,0.752117,0.752049,0.753517,0.750047,0.0,-0.000024,0.000000,0.000024,0.001170,0.001165,...,50.068922,48.951025,-51.048975,0.005962,0.010275,0.195786,1.172903e+08,13.086324,11.188883,0.000000
75%,0.783644,0.783576,0.786046,0.780665,0.0,0.001853,0.001853,0.001900,0.001554,0.001513,...,63.233880,78.135583,-21.864417,0.241499,0.230694,6.650648,1.374546e+08,14.868315,12.244718,1.000000
max,0.830599,0.830703,0.832778,0.829009,0.0,0.012381,0.012381,0.012518,0.003353,0.004876,...,92.692054,100.000000,-0.000000,1.382432,1.213766,38.490564,2.332063e+08,26.710337,19.065638,1.000000


In [8]:
df.head()

,Date,CADUSD=X_Open,CADUSD=X_Close,CADUSD=X_High,CADUSD=X_Low,CADUSD=X_Volume,CADUSD=X_Change,CADUSD=X_Gain,CADUSD=X_Loss,CADUSD=X_Avg_Gain,...,F_RSI,F_k_percent,F_r_percent,F_MACD,F_MACD_EMA,F_ROC,F_PVT,F_Bollinger_Upper,F_Bollinger_Lower,F_Close_pred_class
0,2019-11-26 00:00:00+00:00,0.751665,0.751705,0.752842,0.750999,0.0,-0.000418,0.000000,0.000418,0.000289,...,62.087549,73.913071,-26.086929,0.006892,0.002249,-0.331855,2.394865e+06,9.188927,8.586074,1
1,2019-11-27 00:00:00+00:00,0.753597,0.753636,0.754205,0.752502,0.0,0.001932,0.001932,0.000000,0.000427,...,67.610216,89.583482,-10.416518,0.019164,0.005663,0.220269,2.768411e+06,9.183159,8.647842,0
2,2019-11-29 00:00:00+00:00,0.753125,0.753029,0.753211,0.751145,0.0,-0.000607,0.000000,0.000607,0.000427,...,62.910674,81.250149,-18.749851,0.025755,0.009711,0.221244,2.710846e+06,9.165730,8.712270,0
3,2019-12-02 00:00:00+00:00,0.752734,0.752791,0.753296,0.751569,0.0,-0.000238,0.000000,0.000238,0.000419,...,57.178300,70.833433,-29.166567,0.027163,0.013223,2.270145,2.505366e+06,9.172619,8.717381,0
4,2019-12-03 00:00:00+00:00,0.751552,0.751710,0.752899,0.750728,0.0,-0.001081,0.000000,0.001081,0.000419,...,45.657569,45.833433,-54.166567,0.019687,0.014522,1.137661,1.963927e+06,9.166840,8.712161,1


## Some commodity tickers

- gold: SB=F
- Crude Oil: CL=F
- Silver: XAG=